## Simple Sentiment Analyzer 

- We will build a seniment analyzer to predict positive or negative sentiment in text

In [2]:
import nltk 
import numpy as np 

from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression 
from bs4 import BeautifulSoup 

wordnet_lemmatizer = WordNetLemmatizer() 

#rstrip() returns a copy of the string with the trailing characters stripped 
stopwords = set(w.rstrip() for w in open('stopwords.txt'))


positive_reviews = BeautifulSoup(open('electronics/positive.review').read())
positive_reviews = positive_reviews.find_all('review_text')

negative_reviews = BeautifulSoup(open('electronics/negative.review').read())
negative_reviews = negative_reviews.find_all('review_text')



C:\Users\Siddarth\Anaconda_main\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Siddarth\Anaconda_main\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [3]:
#We want to have the same size of positive reveiws as negative reviews 

np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

In [4]:
def my_tokenizer(s): 
    s = s.lower() 
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) > 2]
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

word_index_map = {} 
current_index = 0

positive_tokenized = [] 
negative_tokenized = [] 

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [6]:
def tokens_to_vector(tokens, label): 
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens: 
        i = word_index_map[t]
        x[i] += 1 
    x = x/x.sum()
    x[-1] = label
    return (x) 

N = len(positive_tokenized) + len(negative_tokenized)

data = np.zeros((N, len(word_index_map) + 1))
i = 0 
for tokens in positive_tokenized: 
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy 
    i += 1 
    
for tokens in negative_tokenized: 
    xy = tokens_to_vector(tokens,0)
    data[i,:] = xy 
    i += 1 
    
np.random.shuffle(data)

X = data[:, :-1]
Y = data[:, -1]

Xtrain = X[:-100,]
Ytrain = Y[:-100, ]
Xtest = X[-100:,]
Ytest = Y[-100:,]

model = LogisticRegression()
model.fit(Xtrain, Ytrain )

print ("Classification rate:", model.score(Xtest, Ytest))

threshold = 0.5 
for word, index in word_index_map.items(): 
    weight =model.coef_[0][index]
    if weight > threshold or weight < -threshold: 
        print (word, weight)
    

Classification rate: 0.81
you 1.02809303206
've 0.718761740328
sound 0.871813652647
customer -0.686587948775
n't -1.95140140923
feature 0.56047607114
time -0.575352437301
cable 0.634588844091
wa -1.63658347207
unit -0.794908729982
easy 1.66419967644
week -0.700382204525
little 0.837124115315
bit 0.651967177135
doe -1.13104863719
lot 0.728633601866
using 0.525306674384
price 2.61271518905
mouse 0.555875550044
tried -0.787866228372
then -1.08801904046
ha 0.782388166282
perfect 1.0099092082
bad -0.759067810063
company -0.519444107502
month -0.84459246804
pretty 0.735965324222
love 1.20682570611
quality 1.44194276385
picture 0.654460538867
try -0.660118928779
excellent 1.31558517618
money -1.04730320398
space 0.585682602902
video 0.562396754571
item -0.963174630696
recommend 0.613562695967
paper 0.661029488001
speaker 0.832983775257
junk -0.572591580393
hour -0.581210758766
card -0.517200196397
fast 0.802754839538
highly 0.879702309911
buy -0.834864038798
comfortable 0.565222741993
support